In [1]:
import talib as taLib
import pymongo
import pandas as pd
import numpy as np
import nsepy
from datetime import datetime
from datetime import timedelta
from datetime import date
from pymongo import MongoClient
from nsepy import get_history
client=MongoClient("mongodb://localhost:27018/",username="stockadm",password="1qaz@WSX")
db=client["stockWarehouse"]
equityList=db["stocklist"]
stockWarehouse=db["stockWarehouse"]
equityListPd = pd.DataFrame(equityList.find({}))
equityListPd.shape[0]


8425

In [2]:
def MovingAverageAnalysis(df,periodfast,periodslow):
    df['EWMfast']=taLib.EMA(df['Close'].values,periodfast)
    df['EWMslow']=taLib.EMA(df['Close'].values,periodslow)
    tempnp=df.iloc[-1].to_numpy()
    dayssincecutover=0
    if tempnp[16] < tempnp[15]:
        trend=1
    else:
        trend=-1
    for j in range(df.shape[0]-1,2,-1):
        if (datadf.iloc[j,15]-datadf.iloc[j,16])*(datadf.iloc[j-1,15]-datadf.iloc[j-1,16]) < 0 :
            dayssincecutover=datadf.shape[0]-j
            break;
    (macd, macdsignal, macdhist)=(taLib.MACD(df['Close'], fastperiod=periodfast, slowperiod=periodslow, signalperiod=9))
    tempnp=macdhist.iloc[-1]
    if tempnp > 1:
        trendnature='Expanding'
    else:
        trendnature='Contracting'
    #print(df.iloc[-1])   
    return(trend,dayssincecutover,trendnature)

In [3]:
datadf=pd.DataFrame(stockWarehouse.find({'Symbol':'HCLTECH'}).sort('Date',-1).limit(90))
print(datadf.columns)
datadf['Hammer']=(taLib.CDLHAMMER(datadf['Open'], datadf['High'],datadf['Low'],datadf['Close'])) + (taLib.CDLHANGINGMAN(datadf['Open'], datadf['High'],datadf['Low'],datadf['Close']))
print(datadf[datadf['Hammer'] != 0])

Index(['%Deliverble', 'Close', 'Date', 'Deliverable Volume', 'High', 'Last',
       'Low', 'Open', 'Prev Close', 'Series', 'Symbol', 'Trades', 'Turnover',
       'VWAP', 'Volume', '_id'],
      dtype='object')
    %Deliverble   Close       Date  Deliverable Volume    High   Last    Low  \
19       0.5016  855.30 2020-12-07             2665422  859.95  855.0  850.4   
61       0.5154  828.05 2020-10-07             3597370  830.60  829.9  823.0   

     Open  Prev Close Series   Symbol  Trades      Turnover    VWAP   Volume  \
19  858.0      858.50     EQ  HCLTECH   83690  4.539560e+14  854.22  5314265   
61  830.0      829.15     EQ  HCLTECH  118532  5.770534e+14  826.72  6980002   

                         _id  Hammer  
19  5ff41db85933977bf6d2ac96     100  
61  5ff41db85933977bf6d2ac6c     100  


In [6]:
datadf=pd.DataFrame(stockWarehouse.find({'Symbol':'SBIN'}).sort('Date',-1).limit(30))
print(datadf.columns)

Index(['%Deliverble', 'Close', 'Date', 'Deliverable Volume', 'High', 'Last',
       'Low', 'Open', 'Prev Close', 'Series', 'Symbol', 'Trades', 'Turnover',
       'VWAP', 'Volume', '_id'],
      dtype='object')


In [22]:
equityList=db["stocklist"]
stockWarehouse=db["stockWarehouse"]

equityListdf=pd.DataFrame(equityList.find({'State':"A"},{'Issuer Name':1,'_id':0},no_cursor_timeout=True))
equityListdf['trend']=0
equityListdf['daysSinceCutOver']=0
equityListdf['trendnature']=''
equityListdf['singleCandlePattern']=''
equityListdf['RSI']=0
equityListdf['Close']=0
equityListdf['sharesbought']=0
equityListdf['moneyinvested']=0    
equityListdf['profit']=0
equityListdf['Capital']=0
equityListdf['trades']=0
equityListdf['cutoverprice']=0                
equityListdf['OBV']=0
equityListdf['paperumbrella']=0
equityListdf['%Deliverable']=0
equityListdf['High']=0
capital_needed=0
equityListdf['sharesbought']=0
portfolio=np.array([['Symbol','price','number of shares']])
print(equityListdf.columns)
for dt in stockWarehouse.find({'Symbol':'SBIN','Date':{'$gte':datetime(2020,1,1)},'Close':{'$lte':1500},'Close':{'$gte':100}},{'_id':0,'Date':1},no_cursor_timeout=True).sort('Date'):
    profit=0  
    trades=0
    sell=0
    buy=0
    limit=50000
    for symbol in equityListdf['Issuer Name']:
        
        # Check if database has latest sttock info by comparing aginst SBI
        for dt1 in stockWarehouse.find({'Symbol':'SBIN','Date':{'$lte':dt['Date']}},{'_id':0,'Date':1}).sort('Date',-1).limit(1):
            refdate=dt1 # Set Reference Data as last date when SBI Stock Data was update
        for maxdate in stockWarehouse.find({'Symbol':symbol,'Date':{'$lte':dt['Date']}},{'_id':0,'Date':1}).sort('Date',-1).limit(1):
            if maxdate == refdate :     
                ################## Get Stock info for last 90 days########################
                datadf=pd.DataFrame(stockWarehouse.find({'Symbol':symbol,'Date':{'$lte':dt['Date']}}).sort('Date',-1).limit(30))
                datadf.set_index('Date',inplace=True)
                datadf.sort_index(inplace=True)
                #print(datadf.iloc[-1].index,end="|")
                #print(datadf.iloc[-1,0])
                ##################Add Last close price to equityListdf
                equityListdf.loc[equityListdf['Issuer Name'] == symbol,'Close']=datadf.iloc[-1,1]
                equityListdf.loc[equityListdf['Issuer Name'] == symbol,'%Deliverable']=datadf.iloc[-1,0]
                equityListdf.loc[equityListdf['Issuer Name'] == symbol,'High']=datadf.iloc[-1,4]
                tempnp=equityListdf.loc[equityListdf['Issuer Name'] == symbol].to_numpy() # Conveting to numpy to use in if conditions
                #print(equityListdf.columns)
                if tempnp[0,6] < 1500 and tempnp[0,6] > 100 and tempnp[0,15] > .50:  # Column 6 is close price columnm 15 is deliverable % 
                    (a,b,c)=MovingAverageAnalysis(datadf,9,21)
                    equityListdf.loc[equityListdf['Issuer Name'] == symbol,'trend']=a
                    equityListdf.loc[equityListdf['Issuer Name'] == symbol,'daysSinceCutOver']=b
                    equityListdf.loc[equityListdf['Issuer Name'] == symbol,'trendnature']=c
                    equityListdf.loc[equityListdf['Issuer Name'] == symbol,'RSI']=taLib.RSI(datadf['Close'],14)[-1]
                    equityListdf.loc[equityListdf['Issuer Name'] == symbol,'OBV']=taLib.OBV(datadf['Close'],datadf['Volume'])[-1]
                    equityListdf['paperumbrella']=(taLib.CDLHAMMER(datadf['Open'], datadf['High'],datadf['Low'],datadf['Close']) + taLib.CDLHANGINGMAN(datadf['Open'], datadf['High'],datadf['Low'],datadf['Close']))[-1]
                    
                    tempnp=equityListdf.loc[equityListdf['Issuer Name'] == symbol].to_numpy()
                    
                    if capital_needed < limit and ((tempnp[0,5] > 60 and tempnp[0,2] < 5   and tempnp[0,1] == 1 and tempnp[0,3] == 'Expanding') or (tempnp[0,14] !=0 and tempnp[0,1] == -1))and tempnp[0,7] == 0 and tempnp[0,13] > 0:
                        for cls in stockWarehouse.find({'Symbol':symbol,'Date':{'$gt':dt['Date']}},{'_id':0,'Open':1,'Date':1}).sort('Date',1).limit(1):
                                buyprice=(cls['Open'] * 1.005)
                        print(tempnp[0,15])        
                        tempnp[0,6]=buyprice
                        tempnp[0,7]=tempnp[0,7] + round(2500/tempnp[0,6])
                        equityListdf.loc[equityListdf['Issuer Name']==symbol,'sharesbought']=tempnp[0,7]
                        equityListdf.loc[equityListdf['Issuer Name']==symbol,'moneyinvested']=tempnp[0,8] + (tempnp[0,6]*round(2500/tempnp[0,6]))
                        capital_needed=capital_needed+(tempnp[0,6]*round(2500/tempnp[0,6]))
                        buy=buy+1
                        portfolio=np.vstack([portfolio,[[symbol,tempnp[0,6],tempnp[0,7]]]])
                    else:
                        if tempnp[0,7] > 0:
                            avgprice=tempnp[0,8]/tempnp[0,7]
                            if tempnp[0,5] < 45 or tempnp[0,1] == -1 or tempnp[0,3] == 'Contracting' :
                                if (avgprice-tempnp[0,6])/avgprice > .1 :
                                    loss=((tempnp[0,6] * tempnp[0,7]) - tempnp[0,8] - 15)
                                if (tempnp[0,16]-avgprice)/avgprice > .05:
                                    loss=((tempnp[0,16] * tempnp[0,7]) - tempnp[0,8] - 15)
                                limit=limit+loss
                                #print(tempnp,loss,tempnp[0,8]*.1)
                                equityListdf.loc[equityListdf['Issuer Name']==symbol,'profit']=tempnp[0,9] + loss
                                capital_needed=(capital_needed-(tempnp[0,6] * tempnp[0,7]))
                                #print('selling shares ',symbol,' @ price',tempnp[0,6],' number ',tempnp[0,7])
                                equityListdf.loc[equityListdf['Issuer Name']==symbol,'sharesbought']=0
                                equityListdf.loc[equityListdf['Issuer Name']==symbol,'trades']=tempnp[0,11]+1
                                equityListdf.loc[equityListdf['Issuer Name']==symbol,'moneyinvested']=0
                                portfolio=portfolio[portfolio[:,0]!=symbol]
                                sell=sell+1

    print(datetime.time(datetime.now()),'Date',dt['Date'].date(),'Profit:',round(equityListdf['profit'].sum()),'Capital:',round(capital_needed),'sellsigs:',sell,'buysigs',buy)
print(limit)
                                

                    
                   

Index(['Issuer Name', 'trend', 'daysSinceCutOver', 'trendnature',
       'singleCandlePattern', 'RSI', 'Close', 'sharesbought', 'moneyinvested',
       'profit', 'Capital', 'trades', 'cutoverprice', 'OBV', 'paperumbrella',
       '%Deliverable', 'High'],
      dtype='object')
0.5807
0.7992
0.5361
0.7487
0.6861


KeyboardInterrupt: 

In [15]:
print(equityListdf.columns)

Index(['Issuer Name', 'trend', 'daysSinceCutOver', 'trendnature',
       'singleCandlePattern', 'RSI', 'Close', 'sharesbought', 'moneyinvested',
       'profit', 'Capital', 'trades', 'cutoverprice', 'OBV'],
      dtype='object')
